# 7.1 Discriminant functions

In the previous chapter, we explored how to extract a feature vector $\textbf{x} = [x_1, x_2, \ldots, x_n]$ to describe an image object or *region*. In this chapter, our goal is **to classify the object as belonging to one of $M$ object classes or categories**, according to said feature vector $\textbf{x}$. In other words, we pursuit the design of a mapping between features and categories $ \textbf{x}^n \rightarrow C$ where $C \in \{C_1,\dots, C_{M}\}$ (see Fig 1.). $\\[15pt]$

<center>
<div>
    <div style="display:inline-block; width:50%;">
        <img src="./images/object_recognition_coco.png" >$\\[3pt]$
    </div>
    <div style="display:inline-block; width:40%;vertical-align:top;margin-left:10px;">
        Scenario: A computer vision system in charge of classifying the objects within kitchens. The considered categories are: $C_1=\text{fridge}$, $C_2=\text{bowl}$, $C_3=\text{microwave}$, $C_4=\text{oven}$, $C_5=\text{toaster}$, $C_6=\text{spoon}$.
        <br /><br />
        Results of object classification: <br /><br />
        $$
        \begin{array}
        _x^1 = [x^1_1, x^1_2, \ldots, x^1_n] \rightarrow C^1 = C_4 \\
        x^2 = [x^2_1, x^2_2, \ldots, x^2_n] \rightarrow C^2 = C_1 \\
        x^3 = [x^3_1, x^3_2, \ldots, x^3_n] \rightarrow C^3 = C_3 \\
        x^4 = [x^4_1, x^4_2, \ldots, x^4_n] \rightarrow C^4 = C_5 \\
        \end{array}
        $$
    </div>    
</div>
    <figcaption>Fig 1. Example of object classification within the COCO dataset.</figcaption>
</center>$\\[1pt]$

For that, the space defined by the features in $\textbf{x}$, also referred as **feature space**, is divided into prediction regions. The image below shows an example of those regions in a scenario with categories $C=\{C_1,C_2,C_3\}$, so $M=3$, and features $\textbf{x}=[x_1,x_2]$:
$\\[10pt]$

<center>
    <img src="./images/clas-space.png" width="300">
    <figcaption>Fig 2. Example of a feature space with three regions.</figcaption>
</center>$\\[3pt]$

Two main steps are needed in order to build an object recognition system and work with it:

- A **training (design) phase**, where sample vectors of known objects are used to learn the classifier (supervised learning).
- A **prediction (online) phase**, where the image objects are classified as belonging to one of the classes based on the learned prediction models.

Such prediction models come in two flavors:

- **Statistical classifiers:**
  - It is assumed that the feature vectors $\textbf{x}$ of the classes $C$ follow a **statistical distribution**.
  - The parameters of such distribution need to be **learned from known objects**.
  - Examples: Naïve Bayessian Classifier, Logistic Regression, Conditional Random Fields.  
- **Non-statistical classifiers:**
  - No assumption is made on the statistical distribution of the feature vector.
  - The coefficient of deterministic discriminant functions are learned.
  - Examples: Support Vector Machine, Perceptron, AdaBoost, Neural Networks, Genetic algorithms.

This notebook focuses on non-statistical classifiers. Concretely it covers:

- Linear discriminant functions (<a href="#711">section 7.1.1</a>).
- The concept of separability (<a href="#712">section 7.1.2</a>).
- Generalized discriminant functions (<a href="#713">section 7.1.3</a>).

All these methos are under the Machine Learning (ML) umbrela. If you want to give a try to Deep Learning (DL) ones, you can do it in notebook *7.4 Deep Learning methods for object detection*!

## Problem context - Traffic sign recognition

Before self-driving vehicles can truly operate autonomously, they would need to identify traffic signs like "speed limit", "school ahead" or "turn ahead". This problem is called **Traffic Sign Recognition (TSR)** and is faced by companies like Tesla or Google (Waymo), and now by *AliquindoiCars*, a startup located in the Andalusia Technology Park in Málaga.

TSR is part of the features collectively called *Advanced Driver Assistance Systems (ADAS)*. It uses image processing techniques to detect the traffic signs. The detection methods used can be generally divided into color based, shape based, or those using low level features. $\\[3pt]$

<img src="./images/signs.jpg" width="400">$\\[3pt]$

Given your growing experience in computer vision, *AliquindoiCars* contacted you asking for a TSR technique to be integrated into a self-driving car. 

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 8.0)

images_path = './images/'

### The data

Since most traffic signs in Spain (and in the rest of the world) have a rectangular, circular or triangular shape, they provided you 3 folders containing 20 images each;

- `./images/circles/{0-19}.png`
- `./images/squares/{0-19}.png`
- `./images/triangles/{0-19}.png`

These folders contain segmented and binarized images of traffic signs captured by a camera in a car. **Our task is to design a recognition system able to distinguish each type of shape**.

Let's take a look at some of these images:

In [ ]:
# Load images
img_circle = cv2.imread(images_path + "circles/0.png", 0)
img_square = cv2.imread(images_path + "squares/0.png", 0)
img_triangle = cv2.imread(images_path + "triangles/0.png", 0)

# Show them!
plt.subplot(131)
plt.imshow(img_circle,cmap='gray');
plt.title('Circular shape')

plt.subplot(132)
plt.imshow(img_square,cmap='gray');
plt.title('Rectangular shape')

plt.subplot(133)
plt.imshow(img_triangle,cmap='gray');
plt.title('Triangular shape');

### Computing feature vectors

Prior to describe the methods that will be used to perform recognition, let's build the feature vectors they are going to use. For this, and given your experience, you are going to use the Hu Moments, a 7-size feature vector that looks like $\textbf{x} = [v_1, v_2, v_3, v_4, v_5, v_6, v_7]$.

### **<span style="color:green"><b><i>ASSIGNMENT 1: Computing the features</i></b></span>**

**Your first task is** to obtain a $20x7$ matrix for each type of shape, where rows index images and columns Hu moments. In this way, the $5^{th}$ row in the matrix characterizing rectangular signs would contain the Hu moments computed from the $5^{th}$ image with that shape. 

In order to get a first-hand impression of how these features are distributed in the feature space for the different shapes, plot the results using [`plt.scatter()`](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.scatter.html) (show only the firsts 2 Hu moments).

In [ ]:
# Assignment 1
def image_moments(region):
    """ Compute moments of the external contour in a binary image.   
    
        Args:
            region: Binary image
                    
        Returns: 
            moments: dictionary containing all moments of the region
    """   
    
    # Get external contour
    contours,_ = cv2.findContours(region,cv2.RETR_EXTERNAL ,cv2.CHAIN_APPROX_NONE)
    cnt = contours[0]
    
    # Compute moments
    moments = cv2.moments(cnt)
    
    return moments

In [ ]:
# Compute Hu moments for circle shape
hu_circles = np.zeros((20,7))

for i in range(20):
    img = cv2.imread(images_path + "circles/" + str(None) + ".png", 0)
    moments = image_moments(None)
    hu_circles[None,None] = cv2.HuMoments(None).flatten()

# Compute Hu moments for square shape
hu_squares = np.zeros((20,7))

for i in range(20):
    img = cv2.imread(images_path + "squares/" + str(None) + ".png", 0)
    moments = image_moments(None)
    hu_squares[None,None] = cv2.HuMoments(None).flatten()

# Compute Hu moments hor triangle shape
hu_triangles = np.zeros((20,7))

for i in range(20):    
    img = cv2.imread(images_path + "triangles/" + str(None) + ".png", 0)
    moments = image_moments(None)
    hu_triangles[None,None] = cv2.HuMoments(None).flatten()

# Define plot axis
plt.axis([0.15, 0.2, -0.0005, 0.0015])

# Plot firsts two Hu moments
plt.xlabel("First Hu moment")
plt.ylabel("Second Hu moment")
plt.scatter(hu_triangles[None,None],hu_triangles[None,None],marker="^")
plt.scatter(hu_circles[None,None],hu_circles[None,None], marker="o")
plt.scatter(hu_squares[None,None],hu_squares[None,None], marker="s");

### Interlude

**You must save the computed Hu moments of the segmented data**, since we are going to need them in the next notebook. For saving numPy matrices, you can use [`np.save()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.save.html) (data is saved at `./data/` by default).

In [ ]:
np.save("./data/hu_circles.npy",hu_circles)
np.save("./data/hu_triangles.npy",hu_triangles)
np.save("./data/hu_squares.npy",hu_squares)

## 7.1.1 Linear discriminant functions <a id="711"></a>

Taking a look at the scatter plot resultant from the previous assignment, we can see how the three types of signs could be segmented using 2 lines. For example: $\\[10pt]$

<center>
    <img src="./images/plot_linear.jpg" width="350">$\\[3pt]$
    <figcaption>Fig 3. First two moments of the binarized traffic signs and a possible division into regions.</figcaption>
</center>

And that's what **linear discriminant functions** do! 

Let's start by describing how such lines could be represented. As we know, a way to define a 2D line is:

$\hspace{2cm} 0 = Ax + By + C\\[3pt]$  
Using another notation:$\\[5pt]$  
$\hspace{2cm}0 = w_1x_1 + w_2x_2 + w_3$$\\[3pt]$  

Note that we can generalize this 2D line to any dimension hyperplane:$\\[5pt]$  
$\hspace{2cm}0 = w_1x_1 + w_2x_2 + \ldots + w_nx_n + w_{n+1}1\ =\ \sum_{i=1}^{n+1}w_ix_i$$ \ =\ w^T\cdot x\\[1pt]$  
In this way, we can divide the feature space for any dimension (e.g. n = 7 for Hu moments).

### Working with two classes

If we are considering just two classes $C=\{C_1,C_2\}$ (*e.g.* an object in a kitchen could be a spoon or a fork), we can define a linear discriminant function as:

$\hspace{2cm} d(\mathbf{x}) = \mathbf{w}^T \cdot \mathbf{x} \ \ \ \ \ \text{(dot product of both vectors)}$

Formally, the vector $w$ is called **weight vector** and we will see a way to compute it automatically from a set of data called **training data**.

In this way, when we have a new object characterized by $\mathbf{x'}$ to classify, $d(\mathbf{x'})$ is computed and:
- If $d(\mathbf{x'})\ge 0$ the new object is assigned $C_1$, and
- if $d(\mathbf{x'})\lt 0$ the object is assigned to $C_2$.

We say that the linear discriminant function $d(\mathbf{x'})$ works as a **decision boundary** between both classes. The following figure illustrates this (blue dots represent characterized objects belonging to $C_1$, while red ones those belonging to $C_2$). $\\[6pt]$

<center>
<img src="./images/linear_discriminant_function_2_categories.png" width="350">
    <figcaption>Fig 3. A decision boundary defined by a decision function that divides the feature space into two regions.</figcaption>
</center>

### **<span style="color:green"><b><i>ASSIGNMENT 2: Classifying samples</i></b></span>**

Let's play a bit with a toy example of a linear discriminant function. Consider that the learned weights during the training phase of the classifier are $\mathbf{w} = [1,0,-3]$, so the linear discriminant function looks like $d(\mathbf{x}) = 1x_1 + 0x_2 -3$. 

**Your task is to:** 

- classify some 2D data using the previous discriminant function (for that, use a combination of [`np.dot()`](https://numpy.org/doc/stable/reference/generated/numpy.dot.html) and [`np.sign()`](https://numpy.org/doc/stable/reference/generated/numpy.sign.html)),  and
- plot them with different colors depending on their assigned class using [`plt.scatter()`](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.scatter.html) and its `c` argument. 

*Tip: [how to color a data point using its class](https://stackoverflow.com/questions/43579626/pandas-plot-with-positive-values-one-color-and-negative-values-another)*.

In [ ]:
# Define weight vector
w = np.array([[1],[0],[-3]])

# Define data matrix
data = np.array([[2,2,1],[1,2,1],[2,1,1],[5,4,1],[4,5,1],[4,4,1]])

# Compute class for each data
classification = np.dot(None,None)
class_ = np.sign(None)

# Plot classified data
plt.scatter(None,None,c=None.flatten(), cmap="bwr")

# Plot discriminant line
x = np.linspace(3, 3, 1000)
y = np.linspace(0, 5, 1000)
plt.plot(x,y);

### Scenarios with N-classes

The previous idea can be generalized to an arbitrary number of classes, defining in that case hyper-planes in a n-dimensional space:

$\hspace{2cm}d_i(\mathbf{x}) = w^i_1x_1 + w^i_2x_2 + \ldots + w^i_nx_n + w^i_{n+1}\ =\ \mathbf{w^i}^T\cdot \mathbf{x}\\[1pt]$  

In this case, the object is assigned to the category which discriminant function returns the highest value. That is:

<img src="./images/linear_discriminant_functions_general_case_low.png">$\\[3pt]$

When more than 2 classes are considered, **the discriminant functions are no longer the decision boundaries between classes**. However, the decision boundary between classes $C_i$ and $C_j$ can be computed as:

$\hspace{2cm}d_{ij}(\mathbf{x}) = d_i(\mathbf{x})-d_j(\mathbf{x})=(\mathbf{w}_i^T-\mathbf{w}_j^T) \cdot \mathbf{x} = \mathbf{w}_{ij}^T \cdot \mathbf{x}$

The following image shows an example with 3 classes (notice that it also exists a decision boundary between classes 1 and 3, $d_{13}(\mathbf{x})$, but it has been omitted for clarity).$\\[10pt]$

<center>
<img src="./images/2lines-discriminant.png"  width="350">
<figcaption>Fig 4. Decision boundaries between 3 categories.</figcaption>
</center>

## 7.1.2 Separability <a id="712"></a>

An important concept regarding object recognition is such of **separability**. In this way, two sets of points defining two classes may be separable or not in a given dimension (e.g. $x_1$, $x_2$, etc.).  

Two sets are **linearly separable** if it exists at least one line in the plane that leaves all the points belonging to one class on one side of the line and all those belonging to the second class on the other side. This idea immediately generalizes to higher-dimensional Euclidean spaces if the line is replaced by a hyperplane. 

Some illustrative examples of this:

<img src="./images/separability-2.png" >$\\[3pt]$

- As you can see, those classes are separable. Also, if only dimension $x_1$ is used, we can separate the classes as well. This is not recomended because there isn't a clear separation using only $x_1$, and $x_2$ is also **discriminative**, that is, it also provides valuable information for separating both classes.

<img src="./images/separability-1.png" >$\\[3pt]$

- In this example the classes are not separable as they overlap in both dimensions. Additionally, we can see that $x_1$ is not discriminative as it doesn't provide any information for separating those classes. The addition of a third dimension/feature could fix this!

<img src="./images/separability-3.png" >$\\[3pt]$

- This is an example of a non-linear separable problem. The classes can't be separated using a line, but a **generalized discriminant function** could do it. We will see this later in this notebook.

### Back to sign shape recognition

Back to our traffic sign shape recognition problem, **we are trying to separate** the usual shapes of traffic signs using the Hu moments. As a first approximation to this problem, we plotted the first and second Hu moments ($x_1$ and $x_2$) in order to analyze them. You should have obtained something like this:

<center>
     <img src="./images/separability-4.png" width="500">$\\[3pt]$
    <figcaption>Fig 5. Result of plotting the two first Hu moments computed from the images in our train data.</figcaption>
</center>

### <font color="blue"><b><i>Thinking about it (1)</i></b></font>

**Now you are in a good position to answer these questions:**

- Is this problem linearly separable?

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
    
- Are the classes separable using only the first Hu moment ($x_1$)?

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
    
- Are the classes separable using only the second Hu moment ($x_2$)?

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
    
- Which dimension is more discriminative?

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
    
- Would be *AliquindoiCars* guys happy if we propose a solution for shape classification based on linear discriminant functions?
  
    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>    

## 7.1.3 Generalized discriminant function <a id="713"></a>

As we saw in the previous section, some classes are not separable using a linear discriminant function. In those cases, we may need a **linear basis function model**, also called **generalized discriminant function**, which permit us to use non-linear functions to separate classes (notice that, despite of this, the problem may still not be separable).

The idea behind these discriminant functions is to transform the $\textbf{x}$ space into a $\textbf{x' = f(x)}$ space ($dim(x) \lt dim(x')$):$\\[5pt]$

$$d(x) = w_1\ \color{blue}{f_1(x)} + w_2\ \color{blue}{f_2(x)} + \ldots + w_k\ \color{blue}{f_k(x)} + w_{k+1} = \sum_{i=1}^{k+1}w_i\ \underbrace{\color{blue}{f_i(x)}}_{\scriptsize\text{New space}}=\mathbf{w}^T\cdot \mathbf{f}(\mathbf{x})$$

$$d(x) = w_1\ \color{blue}{x_1'} + w_2\ \color{blue}{x_2'} + \ldots + w_k\ \color{blue}{x_k'} + w_{k+1} = \sum_{i=1}^{k+1}w_i\ \color{blue}{x_i'} = w^T \cdot \color{blue}{x'}=d'(\mathbf{x}')$$

Thereby:

$$
\left.\begin{aligned}
        \mathbf{x'} = \mathbf{f}(\mathbf{x}) &= [f_1(\mathbf{x}) \ \ \ f_2(\mathbf{x}) \ \ \ \cdots \ \ \ f_k(\mathbf{x}) \ \ \  1]^T\\
        \mathbf{w} &= [w_1 \ \ \ w_2 \ \ \ \cdots \ \ \ w_k \ \ \ w_{k+1}]^T
       \end{aligned}
 \right\}
 \ \  d(\mathbf{x})= \mathbf{w}^T \cdot \mathbf{x} = \mathbf{w}^T \cdot \mathbf{f}(\mathbf{x})
$$
 
 being $\mathbf{f}(\mathbf{x})$ the basis functions!

### **<span style="color:green"><b><i>ASSIGNMENT 3: Transforming between spaces</i></b></span>**

Let's practice this transformation between spaces. **What to do?** You have to complete the method `transform_space()`, which transforms a set of 2D points into a new space defined by:

$$x' = [x_1^2, \ x_1x_2, \ x_2^2, \ x_1, \ x_2, \ 1]$$

The input set of points is a matrix with shape $(n\_data,2)$, and the output is another matrix with shape $(n\_data,6)$, being $n\_data$ the number of points in the set.

In [ ]:
# Assignment 3
def transform_space(data_points):
    """ Compute moments of the external contour in a binary image.   
    
        Args:
            data_points: Input matrix containing a set of 2D points
                    
        Returns: 
            fx: set of points transformed to a quadratic space
    """  
    
    n_data = data_points.shape[None]
    
    fx = np.zeros((None,None))
    
    for i in range(n_data):
        
        x1 = data_points[None,None]
        x2 = data_points[None,None]
        
        fx[i,:] = [None, None, None, None, None, None]
        
    return fx

You can use next code to **test if the results are correct**:

In [ ]:
data = np.array([[2,3],[1,2],[2,7],[2,4],[1,5],[2,4]])

x = transform_space(data)

print(x)

<font color='blue'>**Expected output:**  </font>

     [[ 4.  6.  9.  2.  3.  1.]
     [ 1.  2.  4.  1.  2.  1.]
     [ 4. 14. 49.  2.  7.  1.]
     [ 4.  8. 16.  2.  4.  1.]
     [ 1.  5. 25.  1.  5.  1.]
     [ 4.  8. 16.  2.  4.  1.]]

### **<span style="color:green"><b><i>ASSIGNMENT 4: Classifying in the transformed space</i></b></span>**

Finally, we will see a toy example of a non-linear discriminant function in action. Concretely, we will consider a quadratic function, $d(x)=x_1^2-5x_2$ (so $k=5$), in a two classes problem with $\mathbf{x}=[x_1 \ \ x_2]$. Thereby, the vector of features in the new space is:

$\hspace{2cm}  \mathbf{x'}=[x_1^{'},\dots,x_6^{'}]=[x_1^2 \ \ x_1x_2 \ \ x_2^2 \ \ x_1 \ \ x_2 \ \ 1]^T $ <br />
so:
- $x_1^2=f_1(\mathbf{x})=x_1'$
- $x_1 x_2=f_2(\mathbf{x})=x_2'$
- and so on.

and the vector of weights turns out to be:

$\hspace{2cm} \mathbf{w}=[1,0,0,0,-5,0]$.

As for the new discriminant function we get:

$\hspace{2cm} d(\mathbf{x}) = w_1 \cdot x_1' + w_5 \cdot x_5'$

**Your task is to classify some 2D data** using the previous discriminant function and plot them coloring each point with a color depending on its class. For that, you can rely on the [`np.dot()`](https://numpy.org/doc/stable/reference/generated/numpy.dot.html), [`np.sign()`](https://numpy.org/doc/stable/reference/generated/numpy.sign.html)), and [`plt.scatter()`](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.scatter.html) functions previously used in the second assignment. 

*Tip: Remember to transform the data space before applying the dot product.*

Expected results:

<center>
     <img src="./images/classification_in_the_transformed_space.png">$\\[3pt]$
    <figcaption>Fig 6. Classification of a number of samples in both, the original space and the transformed one.</figcaption>
</center>

In [ ]:
# Assignment 4

# Define weight vector
w = np.array([[1],[0],[0],[0],[-5],[0]])

# Define data matrix
data = np.array([[2,0.5],[1.5,0.25],[3,1],[1,1],[1,0.5],[2,1],[5,5.3],[4,3],[3,2]])

# Trasform from the x space to a f(x) space
transformed_data = transform_space(None)

# Compute class for each data
classification = np.dot(None,None)
class_ = np.sign(None)

# Plot classified data
matplotlib.rcParams['figure.figsize'] = (15.0, 15.0)
ax = plt.subplot(121)
ax.set_aspect('equal')
plt.title('Original space')
plt.xlabel("x_1")
plt.ylabel("x_2")
plt.axis([-0.5, 6.5, -0.5, 7.5])
plt.scatter(None,None,c=None.flatten(), cmap="bwr")

# Plot discriminant line
x = np.linspace(0, 6, 1000)
y = (w[0]*(x**2) + w[5])/-w[4]
plt.plot(x,y);

# Or equivalently plotting the data in the transformed space, 
# where the descision boundary defined by the discrimiant function defines a line
ax = plt.subplot(122)
ax.set_aspect('equal')
plt.title('Transformed space')
plt.xlabel("x'_1")
plt.ylabel("x'_2")
plt.axis([-1, 26, -10, 15])
plt.scatter(None,None,c=None.flatten(), cmap="bwr")

# Plot discrimant line
x = np.linspace(0, 27, 1000)
y = (w[0]*(x) + w[5])/-w[4]
plt.plot(x,y);


## Conclusion

Well done! This was a mostly theoretical notebook, but it described the basis of object classification. You have learned so far:

- how linear and non-linear discriminant functions work,
- to classify objects in a scenario with an arbitrary number of belonging classes using multiple discriminant functions, and
- to check if a feature space allows for the definition of discriminative classifiers looking at the separability between classes.

In the following notebooks in this chapter we will see how to automatically retrieve the weight vector from a set o training data. Exciting, isn't?